In [5]:
"""
Secure Multilingual Text-to-Speech (Hindi/English/Gujarati) + Player + Sweet Girl Preset + Logging
-----------------------------------------------------------------------------------------------
Hardening adds:
  • File security: size limits, extension whitelist, optional AV scan (ClamAV/VirusTotal hook), sandboxable extract
  • Text security: HTML sanitization for UI, max text length, basic content checks
  • Resource limits: max audio duration, clamped sliders (already), truncation safeguards
  • Logging security: UUID IDs (optional), SHA256 integrity hashes, restrictive file/dir permissions
  • UI security: optional basic auth for Gradio, simple per-session rate limiting, safe status rendering
  • Storage hygiene: secure temp handling, permission fixes (chmod), safe pathing

Notes:
  • Replace GRADIO_AUTH_USER/PASS below.
  • Optional AV scan hooks are no-ops unless dependencies/keys exist (won't break runtime).
  • MP3 export still requires FFmpeg on PATH; falls back to WAV automatically.
"""

# ---------------------------
# Imports
# ---------------------------
import os
import io
import csv
import math
import time
import html
import uuid
import hashlib
import tempfile
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import soundfile as sf
from langdetect import detect

# File readers
import PyPDF2
from docx import Document as Docx
from pptx import Presentation

# Audio processing
import librosa
from scipy.signal import butter, sosfilt
from pydub import AudioSegment

# UI
import gradio as gr

# Optional: logs via pandas for convenience (not required for core)
try:
    import pandas as pd
except Exception:
    pd = None

# ---------------------------
# Config & Paths (Security-aware)
# ---------------------------
SPEED_MIN, SPEED_MAX = 0.25, 2.0
GAIN_MIN_DB, GAIN_MAX_DB = -30, 12
PITCH_MIN, PITCH_MAX = -12, 12  # semitones
FREQ_MIN_HZ, FREQ_MAX_HZ = 20, 20000
WAVEL_MIN_M, WAVEL_MAX_M = 0.017, 17.15
AIR_C = 343.0  # m/s

DEFAULTS = {
    "speed": 1.0,
    "gain_db": -2.0,
    "pitch_semitones": 5.0,  # sweet girl preset
    "center_hz": 3000.0,
    "wavelength_m": AIR_C / 3000.0,
    "q": 1.0,
    "sr": 24000,
}

# Security limits
MAX_FILE_SIZE_MB = 10
MAX_FILE_SIZE = MAX_FILE_SIZE_MB * 1024 * 1024
MAX_TEXT_CHARS = 20_000
MAX_AUDIO_SECONDS = 15 * 60  # 15 minutes
SECURE_IDS = True  # set False to use incremental IDs
SANITIZE_DISPLAY = True

# Optional Gradio auth (set user/pass below or keep None to disable)
GRADIO_AUTH_USER = "SOU"
GRADIO_AUTH_PASS = "Ideathon5.0"

BASE_TEXT_DIR = Path("Textual Data")
BASE_AUDIO_DIR = Path("Audio")
LOGS_DIR = Path("logs")
LOG_CSV = LOGS_DIR / "session_log.csv"

LANG_MAP = {
    "hi": "Hindi",
    "en": "English",
    "gu": "Gujarati",
}

# Ensure directories with restrictive permissions
for root in [BASE_TEXT_DIR, BASE_AUDIO_DIR, LOGS_DIR]:
    root.mkdir(parents=True, exist_ok=True)
    try:
        os.chmod(root, 0o700)
    except Exception:
        pass
for lang_name in ["Hindi", "English", "Gujarati"]:
    (BASE_TEXT_DIR / lang_name).mkdir(parents=True, exist_ok=True)
    (BASE_AUDIO_DIR / lang_name).mkdir(parents=True, exist_ok=True)
    for sub in [BASE_TEXT_DIR / lang_name, BASE_AUDIO_DIR / lang_name]:
        try:
            os.chmod(sub, 0o700)
        except Exception:
            pass

# ---------------------------
# Utility: Security helpers
# ---------------------------
ALLOWED_EXTS = {".pdf", ".docx", ".pptx", ".txt"}


def safe_file_check(file_path: str) -> None:
    """Raise on disallowed files; ensure size + extension checks."""
    p = Path(file_path)
    if p.suffix.lower() not in ALLOWED_EXTS:
        raise ValueError("Unsupported file type. Allowed: PDF/DOCX/PPTX/TXT")
    try:
        size = os.path.getsize(file_path)
    except Exception:
        size = 0
    if size <= 0:
        raise ValueError("Empty or unreadable file.")
    if size > MAX_FILE_SIZE:
        raise ValueError(f"File too large (>{MAX_FILE_SIZE_MB} MB).")


def sanitize_for_ui(s: str) -> str:
    return html.escape(s) if SANITIZE_DISPLAY else s


def sha256_bytes(data: bytes) -> str:
    h = hashlib.sha256()
    h.update(data)
    return h.hexdigest()


# Optional AV scan hooks (no-ops if tools/keys missing)
USE_CLAMAV = False  # set True if clamscan/clamd is available
USE_VIRUSTOTAL = False  # set True + provide API key via env VIRUSTOTAL_API_KEY


def av_scan_file(file_path: str) -> str:
    """Return a string result; raise to block. Designed to be safe if AV not present."""
    try:
        if USE_CLAMAV:
            # Example shell call; replace with python-clamd if desired
            import subprocess
            res = subprocess.run(["clamscan", "--no-summary", file_path], capture_output=True, text=True)
            if res.returncode == 1:
                raise ValueError("Antivirus detected a threat in the uploaded file.")
            return res.stdout.strip() or "ClamAV OK"
        if USE_VIRUSTOTAL:
            # Minimal sample (not executed without key); implement as needed
            import json, urllib.request
            api_key = os.getenv("VIRUSTOTAL_API_KEY")
            if not api_key:
                return "VirusTotal disabled (no API key)"
            # VT file scan requires upload; omitted by default to avoid blocking
            return "VirusTotal scan skipped (stub)"
        return "AV scan skipped"
    except Exception as e:
        # Fail-safe: log but don't crash app unless explicit policy to block
        return f"AV scan error: {e}"


# ---------------------------
# Utility: Text Extraction
# ---------------------------

def extract_text(path: str) -> str:
    ext = Path(path).suffix.lower()
    if ext == ".pdf":
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            pages = []
            for i in range(len(reader.pages)):
                try:
                    pages.append(reader.pages[i].extract_text() or "")
                except Exception:
                    pages.append("")
            return "\n\n".join(pages)
    elif ext in {".doc", ".docx"}:
        doc = Docx(path)
        return "\n".join(p.text for p in doc.paragraphs)
    elif ext in {".ppt", ".pptx"}:
        prs = Presentation(path)
        texts = []
        for slide in prs.slides:
            for shape in slide.shapes:
                if hasattr(shape, "text"):
                    texts.append(shape.text)
        return "\n\n".join(texts)
    else:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()


# ---------------------------
# Utility: Chunking & Lang Detect
# ---------------------------

def chunk_text(text: str, max_chars: int = 400) -> list[str]:
    raw_parts = [p.strip() for p in text.replace("\r", "\n").split("\n") if p.strip()]
    chunks = []
    buf = []
    total = 0
    for part in raw_parts:
        if total + len(part) + 1 > max_chars and buf:
            chunks.append(" ".join(buf))
            buf, total = [], 0
        buf.append(part)
        total += len(part) + 1
    if buf:
        chunks.append(" ".join(buf))
    return chunks or ([text[:max_chars]] if text else [])


def detect_lang_safe(text: str) -> str:
    try:
        return detect(text)
    except Exception:
        return "en"


# ---------------------------
# TTS Backends
# ---------------------------
try:
    from TTS.api import TTS as COQUI_TTS
    _HAS_COQUI = True
except Exception:
    _HAS_COQUI = False

try:
    from gtts import gTTS
    _HAS_GTTS = True
except Exception:
    _HAS_GTTS = False

try:
    import pyttsx3
    _HAS_PYTT = True
except Exception:
    _HAS_PYTT = False


def synthesize_chunk(text: str, lang_hint: str, sr: int = DEFAULTS["sr"]) -> np.ndarray:
    lang = lang_hint
    if _HAS_COQUI:
        try:
            model_name = "tts_models/multilingual/multi-dataset/xtts_v2"
            tts = COQUI_TTS(model_name)
            wav = tts.tts(text=text, language=lang if lang in {"en", "hi"} else "en")
            y = np.array(wav, dtype=np.float32)
            if sr != tts.synthesizer.output_sample_rate:
                y = librosa.resample(y, orig_sr=tts.synthesizer.output_sample_rate, target_sr=sr)
            return y.astype(np.float32)
        except Exception:
            pass
    if _HAS_GTTS and lang in {"en", "hi", "gu"}:
        try:
            with tempfile.NamedTemporaryFile(suffix=".mp3", delete=False) as tmp:
                gTTS(text=text, lang=lang).save(tmp.name)
                seg = AudioSegment.from_file(tmp.name)
                seg = seg.set_channels(1).set_frame_rate(sr)
                samples = np.array(seg.get_array_of_samples()).astype(np.float32)
                y = samples / (2 ** (8 * seg.sample_width - 1))
                return y
        except Exception:
            pass
    if _HAS_PYTT:
        try:
            engine = pyttsx3.init()
            for v in engine.getProperty('voices'):
                try:
                    lang_code = (v.languages[0].decode('utf-8') if v.languages else str(v.id))
                except Exception:
                    lang_code = str(v.id)
                if lang in lang_code:
                    engine.setProperty('voice', v.id)
                    break
            with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp:
                out_path = tmp.name
            engine.save_to_file(text, out_path)
            engine.runAndWait()
            y, sr_in = librosa.load(out_path, sr=sr, mono=True)
            try:
                os.unlink(out_path)
            except Exception:
                pass
            return y.astype(np.float32)
        except Exception:
            pass
    return np.zeros(int(sr * 0.5), dtype=np.float32)


def synthesize_text(text: str, sr: int = DEFAULTS["sr"]) -> tuple[np.ndarray, int]:
    text = (text or "")[:MAX_TEXT_CHARS]  # enforce max text length
    chunks = chunk_text(text)
    parts = []
    for ch in chunks:
        lang = detect_lang_safe(ch)
        parts.append(synthesize_chunk(ch, lang, sr=sr))
    y = np.concatenate(parts) if parts else np.zeros(int(sr * 0.5), dtype=np.float32)
    # Normalize
    peak = np.max(np.abs(y)) or 1.0
    y = y / peak * 0.95
    # Enforce max duration
    if len(y) > sr * MAX_AUDIO_SECONDS:
        y = y[: int(sr * MAX_AUDIO_SECONDS)]
    return y.astype(np.float32), sr


# ---------------------------
# Audio FX
# ---------------------------

def apply_gain_db(y: np.ndarray, db: float) -> np.ndarray:
    factor = 10 ** (db / 20.0)
    out = y * factor
    maxv = np.max(np.abs(out))
    if maxv > 1.0:
        out = out / maxv * 0.99
    return out.astype(np.float32)


def apply_speed(y: np.ndarray, sr: int, speed: float) -> np.ndarray:
    speed = max(SPEED_MIN, min(SPEED_MAX, float(speed)))
    return librosa.effects.time_stretch(y, rate=speed).astype(np.float32)


def apply_pitch(y: np.ndarray, sr: int, semitones: float) -> np.ndarray:
    semitones = max(PITCH_MIN, min(PITCH_MAX, float(semitones)))
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=semitones).astype(np.float32)


def bandpass(y: np.ndarray, sr: int, center_hz: float, q: float = 1.0) -> np.ndarray:
    center_hz = float(np.clip(center_hz, FREQ_MIN_HZ, min(FREQ_MAX_HZ, sr/2 - 100)))
    bw = center_hz / max(q, 0.1)
    low = max(10.0, center_hz - bw/2)
    high = min(sr/2 - 10.0, center_hz + bw/2)
    if low >= high:
        return y
    sos = butter(4, [low/(sr/2), high/(sr/2)], btype='bandpass', output='sos')
    return sosfilt(sos, y).astype(np.float32)


# ---------------------------
# Logging & ID helpers (with security)
# ---------------------------

def next_id() -> str:
    """Compute next ID. If SECURE_IDS, use UUID4; else fall back to incremental (6 digits)."""
    if SECURE_IDS:
        return uuid.uuid4().hex  # 32 hex chars
    LOGS_DIR.mkdir(parents=True, exist_ok=True)
    if not LOG_CSV.exists():
        return "000001"
    try:
        with open(LOG_CSV, newline='', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            max_id = 0
            for row in reader:
                try:
                    max_id = max(max_id, int(row.get('ID', 0)))
                except Exception:
                    pass
        return f"{max_id+1:06d}"
    except Exception:
        return "000001"


LOG_HEADERS = [
    "ID", "Language", "InputSource", "Timestamp", "TextFilePath", "AudioFilePath",
    "Speed", "GainDB", "PitchSemitones", "CenterHz", "WavelengthM", "Q",
    "TextSHA256", "AudioSHA256", "AVScan"
]


def write_log_row(row: dict):
    new_file = not LOG_CSV.exists()
    with open(LOG_CSV, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=LOG_HEADERS)
        if new_file:
            writer.writeheader()
    try:
        os.chmod(LOG_CSV, 0o600)
    except Exception:
        pass
    with open(LOG_CSV, "a", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=LOG_HEADERS)
        writer.writerow(row)


def save_text_and_audio(id_str: str, language_name: str, text: str, y: np.ndarray, sr: int, prefer_mp3: bool = True) -> tuple[Path, Path, str, str, str]:
    # Save text
    text_dir = BASE_TEXT_DIR / language_name
    audio_dir = BASE_AUDIO_DIR / language_name
    text_dir.mkdir(parents=True, exist_ok=True)
    audio_dir.mkdir(parents=True, exist_ok=True)

    txt_path = text_dir / f"{id_str}.txt"
    with open(txt_path, "w", encoding="utf-8") as f:
        f.write(text)
    try:
        os.chmod(txt_path, 0o600)
    except Exception:
        pass

    # Save audio as WAV temp first, then convert to MP3 if possible
    wav_tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    sf.write(wav_tmp.name, y, sr)
    wav_tmp.flush()

    audio_path = None
    used_format = "wav"

    if prefer_mp3:
        try:
            seg = AudioSegment.from_file(wav_tmp.name)
            audio_path = audio_dir / f"{id_str}.mp3"
            seg.export(str(audio_path), format="mp3", bitrate="192k")
            used_format = "mp3"
        except Exception:
            # fallback to WAV
            audio_path = audio_dir / f"{id_str}.wav"
            sf.write(str(audio_path), y, sr)
            used_format = "wav"
    else:
        audio_path = audio_dir / f"{id_str}.wav"
        sf.write(str(audio_path), y, sr)
        used_format = "wav"

    try:
        os.unlink(wav_tmp.name)
    except Exception:
        pass

    try:
        os.chmod(audio_path, 0o600)
    except Exception:
        pass

    # Hashes for integrity
    text_hash = sha256_bytes((text or "").encode("utf-8"))
    with open(audio_path, "rb") as af:
        audio_hash = sha256_bytes(af.read())

    return txt_path, audio_path, used_format, text_hash, audio_hash


# ---------------------------
# Session State
# ---------------------------
class Session:
    def __init__(self):
        self.y_full = None
        self.sr = DEFAULTS["sr"]
        self.offset = 0.0
        self.last_processed = None
        self.last_text = ""
        self.last_language = "English"
        self.last_id = None
        self.last_saved_audio_path = None
        self.last_saved_text_path = None
        # Rate limiting: max N processes per minute per session
        self.process_times = []
        self.max_calls_per_minute = 12  # adjust as needed

    def _rate_limit_ok(self) -> bool:
        now = time.time()
        window_start = now - 60
        self.process_times = [t for t in self.process_times if t >= window_start]
        if len(self.process_times) >= self.max_calls_per_minute:
            return False
        self.process_times.append(now)
        return True

    def detect_language_name(self, text: str) -> str:
        code = detect_lang_safe(text or "")
        return LANG_MAP.get(code, "English")

    def load_text(self, text: str):
        # Enforce max text length for processing; keep original for save (truncated)
        self.last_text = (text or "")[:MAX_TEXT_CHARS]
        self.last_language = self.detect_language_name(self.last_text[:400])
        y, sr = synthesize_text(self.last_text, sr=self.sr)
        self.y_full, self.sr = y, sr
        self.offset = 0.0
        self.last_processed = None

    def load_file(self, path: str):
        safe_file_check(path)
        av_result = av_scan_file(path)  # informational
        text = extract_text(path)
        # Attach AV result to status (stored on save)
        self._last_av_result = av_result
        self.load_text(text)

    def process(self, speed=DEFAULTS["speed"], gain_db=DEFAULTS["gain_db"], pitch_semitones=DEFAULTS["pitch_semitones"], center_hz=DEFAULTS["center_hz"], q=DEFAULTS["q" ]):
        if not self._rate_limit_ok():
            raise RuntimeError("Rate limit exceeded. Please wait a moment and try again.")
        if self.y_full is None:
            return None, None
        y = self.y_full.copy()
        # Order: pitch -> speed -> tone -> gain
        if pitch_semitones:
            y = apply_pitch(y, self.sr, pitch_semitones)
        if speed and abs(speed - 1.0) > 1e-3:
            y = apply_speed(y, self.sr, speed)
        if center_hz:
            y = bandpass(y, self.sr, center_hz, q)
        if gain_db:
            y = apply_gain_db(y, gain_db)
        # Enforce max duration again post-FX
        if len(y) > self.sr * MAX_AUDIO_SECONDS:
            y = y[: int(self.sr * MAX_AUDIO_SECONDS)]
        self.last_processed = (y, self.sr)
        return y, self.sr

    def slice_from_offset(self, y: np.ndarray, sr: int) -> np.ndarray:
        start = int(self.offset * sr)
        start = max(0, min(start, len(y)))
        return y[start:]

    def save_run(self, y: np.ndarray, sr: int, freq_hz: float, wavelength_m: float, speed: float, gain_db: float, pitch_semitones: float, q: float, input_source: str = "typed", prefer_mp3: bool = True) -> tuple[str, Path, Path, str]:
        id_str = next_id()
        txt_path, audio_path, fmt, text_hash, audio_hash = save_text_and_audio(id_str, self.last_language, self.last_text, y, sr, prefer_mp3=prefer_mp3)
        timestamp = datetime.utcnow().isoformat()
        av_str = getattr(self, "_last_av_result", "AV scan skipped")
        write_log_row({
            "ID": id_str,
            "Language": self.last_language,
            "InputSource": input_source,
            "Timestamp": timestamp,
            "TextFilePath": str(txt_path),
            "AudioFilePath": str(audio_path),
            "Speed": speed,
            "GainDB": gain_db,
            "PitchSemitones": pitch_semitones,
            "CenterHz": round(freq_hz, 3),
            "WavelengthM": round(wavelength_m, 5),
            "Q": q,
            "TextSHA256": text_hash,
            "AudioSHA256": audio_hash,
            "AVScan": av_str,
        })
        self.last_id = id_str
        self.last_saved_audio_path = audio_path
        self.last_saved_text_path = txt_path
        return id_str, txt_path, audio_path, fmt


SESSION = Session()

# ---------------------------
# Gradio Callbacks (hardened)
# ---------------------------

def ui_load_text(text):
    text = (text or "")[:MAX_TEXT_CHARS]
    SESSION.load_text(text)
    msg = f"Loaded text. Language: {SESSION.last_language}. Length: {len(text)} chars (max {MAX_TEXT_CHARS})."
    return gr.update(value=sanitize_for_ui(msg)), text


def ui_load_file(file):
    if file is None:
        return gr.update(), ""
    try:
        safe_file_check(file.name)
        SESSION.load_file(file.name)
        preview = SESSION.last_text[:500]
        msg = f"Loaded file. Language: {SESSION.last_language}. Text preview below."
        return gr.update(value=sanitize_for_ui(msg)), preview
    except Exception as e:
        return gr.update(value=sanitize_for_ui(f"Load error: {e}")), ""


def ui_process(speed, gain_db, pitch_semitones, center_hz, wavelength, q):
    try:
        # Wavelength <-> Frequency synchronization
        if wavelength is not None:
            wavelength = float(max(WAVEL_MIN_M, min(WAVEL_MAX_M, float(wavelength))))
            center_hz = AIR_C / wavelength
        center_hz = float(max(FREQ_MIN_HZ, min(FREQ_MAX_HZ, float(center_hz))))

        y, sr = SESSION.process(speed, gain_db, pitch_semitones, center_hz, q)
        if y is None:
            return None, sanitize_for_ui("No audio yet."), ""

        # Playback slice from current offset
        sliced = SESSION.slice_from_offset(y, sr)
        play_wav_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
        sf.write(play_wav_path, sliced, sr)

        # Save FULL processed audio + text with auto ID (archive/log)
        full_id, txt_path, audio_path, fmt = SESSION.save_run(y, sr, center_hz, AIR_C / center_hz, speed, gain_db, pitch_semitones, q, input_source="typed", prefer_mp3=True)

        status = (
            f"Saved ID #{full_id} | Lang: {SESSION.last_language} | File: {Path(audio_path).name} ({fmt.upper()})\n"
            f"Text → {txt_path}\nAudio → {audio_path}\n"
            f"Offset: {SESSION.offset:.1f}s | Play length: {len(sliced)/sr:.1f}s | λ≈{AIR_C/center_hz:.3f} m"
        )
        return play_wav_path, sanitize_for_ui(status), str(audio_path)
    except RuntimeError as rexc:
        return None, sanitize_for_ui(str(rexc)), ""
    except Exception as e:
        return None, sanitize_for_ui(f"Process error: {e}"), ""


def ui_seek(delta):
    SESSION.offset = max(0.0, SESSION.offset + float(delta))
    if SESSION.last_processed is None:
        return None, sanitize_for_ui(f"Offset: {SESSION.offset:.1f}s")
    y, sr = SESSION.last_processed
    sliced = SESSION.slice_from_offset(y, sr)
    wav_path = tempfile.NamedTemporaryFile(suffix=".wav", delete=False).name
    sf.write(wav_path, sliced, sr)
    return wav_path, sanitize_for_ui(f"Offset: {SESSION.offset:.1f}s, Play length: {len(sliced)/sr:.1f}s")


def ui_reset_offset():
    SESSION.offset = 0.0
    return ui_seek(0)


# ---------------------------
# Build Gradio Interface
# ---------------------------
with gr.Blocks(title="Secure Multilingual TTS Reader + Player + Logging") as demo:
    gr.Markdown("# Multilingual TTS Reader + Player + Logging (Hardened)\nDefault preset: **sweet girl's voice** (Pitch +5, Tone 3 kHz, Gain −2 dB). Adjust any slider to override.\n\n**Security:** file size limits, text length caps, integrity hashes, optional auth, basic rate limiting. For MP3 export, FFmpeg must be installed.")

    with gr.Row():
        txt = gr.Textbox(label="Paste Text", lines=6, placeholder="Type or paste Hindi / English / Gujarati / mixed text… (max 20k chars)")
        file = gr.File(file_types=[".pdf", ".docx", ".pptx", ".txt"], label="Or upload: PDF / DOCX / PPTX / TXT (≤10 MB)")
    with gr.Row():
        load_text_btn = gr.Button("Load Text")
        load_file_btn = gr.Button("Load File")
        load_status = gr.Textbox(label="Loader Status", interactive=False)

    with gr.Row():
        speed = gr.Slider(SPEED_MIN, SPEED_MAX, value=DEFAULTS["speed"], step=0.05, label="Speed (x)")
        gain = gr.Slider(GAIN_MIN_DB, GAIN_MAX_DB, value=DEFAULTS["gain_db"], step=1, label="Loudness / Gain (dB)")
        pitch = gr.Slider(PITCH_MIN, PITCH_MAX, value=DEFAULTS["pitch_semitones"], step=0.5, label="Pitch Shift (semitones)")
    with gr.Row():
        center = gr.Slider(FREQ_MIN_HZ, FREQ_MAX_HZ, value=DEFAULTS["center_hz"], step=1, label="Tone Center Frequency (Hz)")
        wavelength = gr.Slider(WAVEL_MIN_M, WAVEL_MAX_M, value=DEFAULTS["wavelength_m"], step=0.001, label="Wavelength (m)")
        q = gr.Slider(0.2, 10.0, value=DEFAULTS["q"], step=0.1, label="Tone Q (bandwidth)")

    process_btn = gr.Button("Process + Save + Play from Offset")
    audio = gr.Audio(label="Audio Output", interactive=False)
    status = gr.Textbox(label="Status & Saved Paths", interactive=False)
    last_audio_path = gr.Textbox(label="Last Saved Audio Path", interactive=False)

    with gr.Row():
        back10 = gr.Button("⏪ Rewind 10s")
        ahead10 = gr.Button("⏩ Forward 10s")
        reset = gr.Button("⏮️ Restart")

    # Events
    load_text_btn.click(ui_load_text, inputs=[txt], outputs=[load_status, txt])
    load_file_btn.click(ui_load_file, inputs=[file], outputs=[load_status, txt])
    process_btn.click(ui_process, inputs=[speed, gain, pitch, center, wavelength, q], outputs=[audio, status, last_audio_path])
    back10.click(lambda: ui_seek(-10), outputs=[audio, status])
    ahead10.click(lambda: ui_seek(10), outputs=[audio, status])
    reset.click(ui_reset_offset, outputs=[audio, status])

# In Jupyter, run:
if GRADIO_AUTH_USER and GRADIO_AUTH_PASS:
    demo.launch(debug=False, share=False, auth=(GRADIO_AUTH_USER, GRADIO_AUTH_PASS))
else:
    demo.launch(debug=False, share=False)

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


C:\Users\amanl\anaconda3\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(
